<a href="https://colab.research.google.com/github/NagaSandeepAthota/nathota/blob/main/IT7103_assignment5_yourid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv('/content/real_estate_value.csv')

In [3]:
df = df.dropna()

In [4]:
y = df['UnitPrice']
x = df.drop('UnitPrice', axis=1)

In [5]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state=42)

In [6]:
symmetric_columns = []
skewed_columns = []

for column in x.columns:
    if df[column].skew() > -1 and df[column].skew() < 1:
        symmetric_columns.append(column)
    else:
        skewed_columns.append(column)

In [7]:
print(symmetric_columns)
print(skewed_columns)

['HouseAge', 'NoOfStores', 'Latitude']
['DistanceToMRT', 'Longitude']


In [8]:
symmetric_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy='mean')),
  ('scaler', StandardScaler())
])

skewed_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy='mean')),
  ('log_transform', FunctionTransformer(np.log1p)),
  ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer([
    ('numeric_symmetric', symmetric_pipeline, symmetric_columns),
    ('numeric_skewed', skewed_pipeline, skewed_columns),
])

pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

In [9]:
pipeline.fit(df)
train_transformed = pipeline.transform(train_x)
test_transformed = pipeline.transform(test_x)

In [10]:
decision_tree = DecisionTreeRegressor(random_state=42)
decision_tree.fit(train_transformed, train_y)
decision_pred = decision_tree.predict(test_transformed)

In [11]:
mse_dt = mean_squared_error(test_y, decision_pred)
r2_dt = r2_score(test_y, decision_pred)

In [12]:
random_forest = RandomForestRegressor(random_state=42)
random_forest.fit(train_transformed, train_y)
random_pred = random_forest.predict(test_transformed)

In [13]:
mse_rf = mean_squared_error(test_y, random_pred)
r2_rf = r2_score(test_y, random_pred)

In [14]:
print("Decision Tree Performance")
print("Mean Squared Error (MSE):", mse_dt)
print("R-squared (R2):", r2_dt)
print("\nRandom Forest Performance")
print("Mean Squared Error (MSE):", mse_rf)
print("R-squared (R2):", r2_rf)

Decision Tree Performance
Mean Squared Error (MSE): 65.39755999999998
R-squared (R2): 0.6089250137675013

Random Forest Performance
Mean Squared Error (MSE): 43.18817382190911
R-squared (R2): 0.7417363203946773
